# Case 2 - Transfer model
Neural Networks for Machine Learning Applications<br>
24.2.2022, Jesse Jyrälä & Lauri Marjanen<br>
Metropolia University of Applied Sciences

# Idea of this model

In [1]:
#Imports
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator

print('Tensorflow version: ', tf.__version__)
# Ensure that installs for GPU running exist
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Tensorflow version:  2.8.0
Num GPUs Available:  0


# Dataset

Fetching data and splitting it into training and validation data. 4 to 1 split seems to work best but 10 to 1 could also be used with good results.

In [2]:
BATCH_SIZE = 32
IMG_SIZE = (128, 128)

In [3]:
curDir = os.getcwd()
data_dir = curDir + '/Case2Data/chest_xray/train'

# data_dir = "D:/Koulu/Datasets/Case2Data/chest_xray/train"
# data_test_dir = "D:/Koulu/Datasets/Case2Data/chest_xray/test"
# data_comb_dir = "D:/Koulu/Datasets/Case2Data/chest_xray/CombinedTestTrain"
# val_data_dir = "D:/Koulu/Datasets/Case2Data/chest_xray/val"
os.listdir(data_dir)

['NORMAL', 'PNEUMONIA']

In [4]:
# Creating datasets for training and validation with 4 to 1 split
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split = 0.2,
  subset = "training",
  seed = 123,
  image_size = IMG_SIZE,
  batch_size = BATCH_SIZE)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split = 0.2,
  subset = "validation",
  seed = 123,
  image_size = IMG_SIZE,
  batch_size = BATCH_SIZE)

IMG_SHAPE = IMG_SIZE + (3,)

Found 5216 files belonging to 2 classes.
Using 4173 files for training.
Found 5216 files belonging to 2 classes.
Using 1043 files for validation.


In [5]:
# # Creating datasets for training and validation with 4 to 1 split
# dataset = tf.keras.preprocessing.image_dataset_from_directory(
#   data_dir,
#   image_size = IMG_SIZE,
#   batch_size = BATCH_SIZE)


In [6]:
# train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# Modeling

First using keras pretarained models to base train the model. Testing ResNet50 and ResNet101

In [7]:
IMG_SHAPE = IMG_SIZE + (3,)

!!Remeber to comment out all pretrained models but the one you want to use!!

In [8]:
#pretrained model with InceptionV3

# preprocess_input = tf.keras.applications.inception_v3.preprocess_input

# base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
#                                                include_top=False,
#                                                weights='imagenet')

# base_model.trainable = False

In [9]:
# #pretrained model with InceptionResNet_V2

# preprocess_input = tf.keras.applications.inception_resnet_v2.preprocess_input

# base_model = tf.keras.applications.InceptionResNetV2(input_shape=IMG_SHAPE,
#                                                include_top=False,
#                                                weights='imagenet')

# base_model.trainable = False

In [10]:
# #pretrained model with ResNet50

preprocess_input = tf.keras.applications.resnet.preprocess_input

base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

In [11]:
#pretrained model with ResNet101

# preprocess_input = tf.keras.applications.resnet.preprocess_input

# base_model = tf.keras.applications.ResNet101(input_shape=IMG_SHAPE,
#                                                include_top=False,
#                                                weights='imagenet')

# base_model.trainable = False

171458560/171446536 [==============================] - 6s 0us/step


In [12]:
#pretrained model with VGG16 or VGG19

# preprocess_input = tf.keras.applications.vgg19.preprocess_input

# base_model = tf.keras.applications.VGG19(input_shape=IMG_SHAPE,
#                                                include_top=False,
#                                                weights='imagenet')

# base_model.trainable = False

In [13]:
#getting layers from keras for the child model to use

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1)

In [14]:
#Modeling the child

inputs = tf.keras.Input(shape = IMG_SHAPE)
holder = preprocess_input(inputs)
holder = base_model(holder, training = False)
holder = global_average_layer(holder)
holder = tf.keras.layers.Dropout(0.2)(holder)
outputs = prediction_layer(holder)
model = tf.keras.Model(inputs, outputs)

In [15]:
#compiling model. EDIT OUT: ei mitään hajuu mitä toi leraning rate tekee

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics=['accuracy'])

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 128, 128, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 128, 128, 3)      0         
                                                                 
 resnet101 (Functional)      (None, 4, 4, 2048)        42658176  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0     

In [17]:
model.trainable_variables

[<tf.Variable 'dense/kernel:0' shape=(2048, 1) dtype=float32, numpy=
 array([[-0.01777567],
        [ 0.00167895],
        [ 0.01812768],
        ...,
        [-0.04235227],
        [-0.02008777],
        [-0.00365984]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

# Training

In [18]:
loss1, accuracy1 = model.evaluate(test_dataset)

print(f"Validation loss:     {loss1:.2f}")
print(f"Validation accuracy: {accuracy1:.2f}")

33/33 [==============================] - 47s 1s/step - loss: 0.6961 - accuracy: 0.4554
Validation loss:     0.70
Validation accuracy: 0.46


In [19]:
trained = model.fit(train_dataset, batch_size=BATCH_SIZE, epochs=5)

Epoch 1/5
131/131 [==============================] - 191s 1s/step - loss: 0.4208 - accuracy: 0.7973
Epoch 2/5
131/131 [==============================] - 186s 1s/step - loss: 0.2534 - accuracy: 0.8893
Epoch 3/5
131/131 [==============================] - 187s 1s/step - loss: 0.2157 - accuracy: 0.9089
Epoch 4/5
131/131 [==============================] - 185s 1s/step - loss: 0.1891 - accuracy: 0.9185
Epoch 5/5
131/131 [==============================] - 185s 1s/step - loss: 0.1819 - accuracy: 0.9231


# Evaluation

In [20]:
loss1, accuracy1 = model.evaluate(test_dataset)

print(f"Validation loss:     {loss1:.2f}")
print(f"Validation accuracy: {accuracy1:.2f}")

33/33 [==============================] - 47s 1s/step - loss: 0.1236 - accuracy: 0.9501
Validation loss:     0.12
Validation accuracy: 0.95


In [21]:

acc = trained.history['accuracy']
val_acc = trained.history['val_accuracy']

loss = trained.history['loss']
val_loss = trained.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')
plt.grid()

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.grid()
plt.show()

KeyError: 'val_accuracy'

# Conclusion